In [3]:
import time
import pandas as pd

In [4]:
# Selenium imports
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [22]:
def init_driver(path):    #path = '../driver/chromedriver.exe'
    # Initialize crome driver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--profile-directory=Default')
#     chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--disable-pligins-discovery')
    chrome_options.add_argument('--start-maximized')
    
    browser = webdriver.Chrome(executable_path=path, chrome_options=chrome_options)
    
    return browser

In [29]:
def search_jobs(job_name:str, job_location:str, num_jobs:int, sleep_time, verbose):
    
    # initialize webdriver
    browser = init_driver(path='../driver/chromedriver.exe') # driver =  browser  path = '../driver/chromedriver.exe'
    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+science&locT=N&locId=96&jobType=&context=Jobs&sc.keyword=data+science&dropdown=0'
    browser.get(url)
    accept_cookies = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id='onetrust-accept-btn-handler']"))).click()

    job_title = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Job Title, Keywords, or Company']")))
    location = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Location']")))
    
    job_title.clear()
    location.clear()
    time.sleep(5)  # wait for 5 seconds
    
    job_title.send_keys(job_name)
    location.send_keys(job_location)
    time.sleep(3)
    
    press_submit = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
#     time.sleep(sleep_time)
    time.sleep(3)
    jobs = []
    
    while len(jobs) < num_jobs:
###################CLOSE SIGN UP POP-UP#############################################        
        time.sleep(sleep_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            browser.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            browser.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass
####################################################################################        
        # Going through each job in this page
        job_boxes = browser.find_elements_by_class_name("jl")
        for job_box in job_boxes:
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            job_box.click()
            time.sleep(1)
            
            collected_successfully = False
            while not collected_successfully:
                try:
                    company_name = browser.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = browser.find_element_by_xpath('.//div[@class="location"]').text
                    job_nu_naam = browser.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = browser.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text  # The role para on webpage
                    collected_successfully = True
                except:
                    time.sleep(5)
                    
                try:
                    salary_estimate = browser.find_element_by_xpath('.//span[@class="css-1uyte9r css-hca4ks e1wijj242"]').text  # css-1uyte9r css-hca4ks e1wijj242
                except NoSuchElementException:
                    salary_estimate = -1
                    
                try:
                    rating = browser.find_element_by_xpath('.//span[@class="rating"]').text
                except NoSuchElementException:
                    rating = -1
                    
                if verbose:
                    print(f"Job Title: {job_nu_naam}")
                    print(f"Salary Estimate: {salart_estimate}")
                    print(f"Job Description: {job_description}")
                    print(f"Rating: {rating}")
                    print(f"Company Name: {company_name}")
                    print(f"Location: {location}")
                
                #Going to the Company tab...
                try:
                    browser.find_element_by_xpath('.//div[@data-tab-type="overview"]//span[text()="Company"]//following-sibling::*').click()
                    try:
                        headquarters = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                    except NoSuchElementException:
                        headquarters = -1
                        
                    try:
                        size = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[@text()="Size"]//following-sibling::*').text
                    except NoSuchElementException:
                        size = -1
                    
                    try:
                        founded = browser.find_element_by_xpath('.div[@class="infoEntity"]//label[@text()="Founded"]//following-siblings::*').text
                    except NoSuchElementException:
                        founded = -1
                    
                    try:
                        type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                    except NoSuchElementException:
                        type_of_ownership = -1
                
                    try:
                        industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                    except NoSuchElementException:
                        industry = -1
              
                    try:
                        sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                    except NoSuchElementException:
                        sector = -1

                    try:
                        revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                    except NoSuchElementException:
                        revenue = -1

                    try:
                        competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                    except NoSuchElementException:
                        competitors = -1

                except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                    headquarters = -1
                    size = -1
                    founded = -1
                    type_of_ownership = -1
                    industry = -1
                    sector = -1
                    revenue = -1
                    competitors = -1
                
                if verbose:
                    print(f"Headquaters: {headquarters}")
                    print(f"Size: {size}")
                    print(f"Founded: {founded}")
                    print(f"Type of Ownership: {type_of_ownership}")
                    print(f"Industry: {industry}")
                    print(f"Sector:{sector}")
                    print(f"Revenue: {revenue}")
                    print(f"Competitors: {competitors}")
                    print("#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$")
                    
                jobs.append({'Job Title':job_nu_naam,
                            "Salary Estimate" : salary_estimate,
                            "Job Description" : job_description,
                            "Rating" : rating,
                            "Company Name" : company_name,
                            "Location" : location,
                            "Headquarters" : headquarters,
                            "Size" : size,
                            "Founded" : founded,
                            "Type of ownership" : type_of_ownership,
                            "Industry" : industry,
                            "Sector" : sector,
                            "Revenue" : revenue,
                            "Competitors" : competitors})
            try:
                browser.find_element_by_xpath('.//li[@class="next"]//a//following-sibling::*').click()
            except NoSuchElementException:
                print(f"Scraping terminated before reaching target of jobs. Needed {num_jobs} , got {len(jobs)}")
        
        return pd.DataFrame(jobs)


In [31]:
df = search_jobs(job_name='data scientist', job_location='California', num_jobs=20, sleep_time=15, verbose=False)
df

C:\Users\Saurabh\anaconda3\envs\webscrape\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


 x out worked
Progress: 0/20
Scraping terminated before reaching target of jobs. Needed 20 , got 1
Progress: 1/20
Scraping terminated before reaching target of jobs. Needed 20 , got 2
Progress: 2/20
Scraping terminated before reaching target of jobs. Needed 20 , got 3
Progress: 3/20
Scraping terminated before reaching target of jobs. Needed 20 , got 4
Progress: 4/20
Scraping terminated before reaching target of jobs. Needed 20 , got 5
Progress: 5/20
Scraping terminated before reaching target of jobs. Needed 20 , got 6
Progress: 6/20
Scraping terminated before reaching target of jobs. Needed 20 , got 7
Progress: 7/20
Scraping terminated before reaching target of jobs. Needed 20 , got 8
Progress: 8/20
Scraping terminated before reaching target of jobs. Needed 20 , got 9
Progress: 9/20
Scraping terminated before reaching target of jobs. Needed 20 , got 10
Progress: 10/20
Scraping terminated before reaching target of jobs. Needed 20 , got 11
Progress: 11/20
Scraping terminated before reach

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist/Applied Mathematician R-19,-1,Data Scientist/Applied Mathematician R-19\nThe...,5.0,"Exquadrum, Inc.\n5.0","Edwards AFB, CA",-1,-1,-1,-1,-1,-1,-1,-1
1,Data Scientist,$125K - $154K (Glassdoor est.),"Are you passionate about using data, finding i...",4.2,TaylorMade Golf\n4.2,"Carlsbad, CA",-1,-1,-1,-1,-1,-1,-1,-1
2,Data Scientist - HCI,-1,"Data Scientist - HCI\nLong Beach, CA\nFull-tim...",2.3,SCAN Group\n2.3,"Long Beach, CA",-1,-1,-1,-1,-1,-1,-1,-1
3,Senior Data Scientist,-1,About Fracta\nFracta.ai usesmachine learning a...,-1,Fracta,"Redwood City, CA",-1,-1,-1,-1,-1,-1,-1,-1
4,Scientist 1,-1,Cascadia is more than a testing lab; we are ro...,-1,"Cascadia Labs California, LLC","Redwood Valley, CA",-1,-1,-1,-1,-1,-1,-1,-1
5,"Research Scientist, Computational Biology",$99K - $158K (Glassdoor est.),Job Description\nThe Research Scientist I will...,3.7,Kite Pharma\n3.7,"Santa Monica, CA",-1,-1,-1,-1,-1,-1,-1,-1
6,Applied Scientist,$20K - $125K (Glassdoor est.),"Company Overview\n\nGauss Labs, specializing i...",3.7,SK hynix America\n3.7,"San Jose, CA",-1,-1,-1,-1,-1,-1,-1,-1
7,Director of Data Insights and Analytics,$99K - $193K (Glassdoor est.),Born from the belief that great things come to...,4.6,Yummly\n4.6,"San Carlos, CA",-1,-1,-1,-1,-1,-1,-1,-1
8,(FOOD SCIENTIST) Sr. Product Development Techn...,-1,Horizon Food Groupis a strategic bakery and sw...,-1,NEMO'S BAKERY,California,-1,-1,-1,-1,-1,-1,-1,-1
9,Data Scientist,$81K - $137K (Glassdoor est.),Apply\n\nDescription\n\nWalton Isaacson is loo...,2.0,Walton Isaacson\n2.0,"Culver City, CA",-1,-1,-1,-1,-1,-1,-1,-1
